In [1]:
from Spec_UI import SpecificationSetupUI

ui = SpecificationSetupUI()

# Known Issue:
# 1. Cell Type Id assignment issue (no need manual assign)
# 2. Save configuration supose to validate/save to json (right now it's just UI)
'''
The "Save Configuration" button might not be needed as .spec_dict validate constraint.

'''
# 3. Plugins grouping are not fully correct | Plugins are not saved
'''
Follow the Widzard simulation? or how should this be grouped?
Data are not stored in the json file as they suppose to --> debug file.
(Stretch HTML goal) Also, the UI can be fixed vertical tabs, tabs in a tabs seems a little weird
'''

# 4. Range constraint rely on defintion and widget min/max adjustment
'''
- Redundency if both front/back check;
- Rely more on the defition of constraint + display that text to user
- Frontend adjustment as user can't select negative value --> min values are usually defined in the API but not max values,
max values might need to rely on defintion check.
'''


"\n- Redundency if both front/back check;\n- Rely more on the defition of constraint + display that text to user\n- Frontend adjustment as user can't select negative value --> min values are usually defined in the API but not max values,\nmax values might need to rely on defintion check.\n"

In [45]:
import anywidget
import traitlets

class EditableTableWidget(anywidget.AnyWidget):
    _esm = """
        export function render({ model, el }) {
            function renderTable() {
                // Add a style tag for the highlight class if not already present
                if (!document.getElementById('editable-table-row-highlight-style')) {
                    const style = document.createElement('style');
                    style.id = 'editable-table-row-highlight-style';
                    style.textContent = `
                        .editable-table-row-highlight {
                            background-color: #f8d7da !important;
                        }
                    `;
                    document.head.appendChild(style);
                }

                el.innerHTML = "";
                const table = document.createElement('table');
                table.style.borderCollapse = "collapse";

                const rows = model.get('rows');
                const headers = model.get('headers') || [];
                const colCount = rows[0]?.length || 0;

                // Header
                const headerRow = document.createElement('tr');
                for (let i = 0; i < colCount; i++) {
                    const th = document.createElement('th');
                    th.textContent = headers[i] || `Column ${i + 1}`;
                    th.style.fontWeight = "600"; // semibold
                    th.style.fontSize = "1.25em"; // slightly larger font
                    th.style.color = "#000"; // black
                    th.style.padding = "6px";
                    th.style.textAlign = "left";
                    headerRow.appendChild(th);
                }
                const actionTh = document.createElement('th');
                actionTh.textContent = "";
                headerRow.appendChild(actionTh);
                table.appendChild(headerRow);

                // Body
                rows.forEach((rowData, rowIndex) => {
                    const row = document.createElement('tr');

                    rowData.forEach((cellData, cellIndex) => {
                        const cell = document.createElement('td');
                        if (headers[cellIndex] === "Frozen") {
                            const input = document.createElement('input');
                            input.type = "checkbox";
                            input.checked = (cellData === true || cellData === "True" || cellData === "true");
                            input.onchange = () => {
                                rows[rowIndex][cellIndex] = input.checked;
                                model.set('rows', rows);
                                model.save_changes();
                            };
                            cell.appendChild(input);
                        } else {
                            const input = document.createElement('input');
                            input.value = cellData;
                            input.style.width = "100px";
                            if (headers[cellIndex] === "Cell Type") {
                                input.style.border = "1px solid #ccc"; // light grey by default
                                input.onfocus = () => {
                                    input.style.border = "1.5px solid #000"; // black on focus
                                };
                                input.onblur = () => {
                                    input.style.border = "1px solid #ccc"; // revert to light grey
                                };
                            }
                            input.oninput = () => {
                                rows[rowIndex][cellIndex] = input.value;
                                model.set('rows', rows);
                                model.save_changes();
                            };
                            cell.appendChild(input);
                        }
                        cell.style.padding = "4px";
                        row.appendChild(cell);
                    });

                    // Delete button
                    const deleteCell = document.createElement('td');
                    const deleteButton = document.createElement('button');
                    deleteButton.textContent = "✖";
                    deleteButton.style.background = "transparent";
                    deleteButton.style.border = "none";
                    deleteButton.style.cursor = "pointer";
                    deleteButton.style.fontSize = "10px";
                    deleteButton.style.padding = "0 8px";
                    deleteButton.style.color = "#AAA";
                    deleteButton.onmouseover = () => {
                        row.classList.add('editable-table-row-highlight');
                        deleteButton.style.color = "#000"; // black on hover
                    };
                    deleteButton.onmouseout = () => {
                        row.classList.remove('editable-table-row-highlight');
                        deleteButton.style.color = "#AAA"; // revert to grey
                    };
                    deleteButton.onclick = () => {
                        rows.splice(rowIndex, 1);
                        model.set('rows', rows);
                        model.save_changes();
                        renderTable();
                    };

                    deleteCell.appendChild(deleteButton);
                    deleteCell.style.textAlign = "center";
                    row.appendChild(deleteCell);

                    table.appendChild(row);
                });

                el.appendChild(table);
            }

            model.on('change:rows', renderTable);
            model.on('change:headers', renderTable);
            renderTable();
        }
    """
    rows = traitlets.List(traitlets.List(traitlets.Unicode())).tag(sync=True)
    headers = traitlets.List(traitlets.Unicode()).tag(sync=True)


In [46]:
table = EditableTableWidget(
    rows=[["Medium", "True"], ["T1", "False"], ["T2", "False"]],
    headers=["Cell Type", "Frozen"]
)
display(table)

EditableTableWidget(headers=['Cell Type', 'Frozen'], rows=[['Medium', 'True'], ['T1', 'False'], ['T2', 'False'…

In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [11]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [13]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
print(volume_widget.__dict__)
# print(volume_widget.widgets)  # See the actual widget objects
# print(ui.plugins_tab.plugin_widgets.keys())

{'plugin_name': 'VolumePlugin', 'plugin_class': <class 'cc3d.core.PyCoreSpecs.VolumePlugin'>, 'default_instance': <cc3d.core.PyCoreSpecs.VolumePlugin object at 0x000001CDDDC4CD00>, 'cell_types': ['Medium', 'RT'], 'widgets': {'active': Checkbox(value=True, description='VolumePlugin', indent=False), 'config_container': VBox(children=(VBox(children=(HBox(children=(Label(value='Medium', layout=Layout(width='120px')), Label(value='Target Volume:', layout=Layout(width='100px')), FloatText(value=21.0, layout=Layout(width='120px')), Label(value='Lambda Volume:', layout=Layout(width='100px')), FloatText(value=2.0, layout=Layout(width='120px'))), layout=Layout(padding='4px 0 4px 12px'), _dom_classes=('volume-row',)), HBox(children=(Label(value='RT', layout=Layout(width='120px')), Label(value='Target Volume:', layout=Layout(width='100px')), FloatText(value=25.0, layout=Layout(width='120px')), Label(value='Lambda Volume:', layout=Layout(width='100px')), FloatText(value=2.0, layout=Layout(width='12

In [2]:
# %run ./Spec_UI.py IGNORE

In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [3]:
import ipywidgets as widgets
from IPython.display import display
from cc3d.core.PyCoreSpecs import PLUGINS

for plugin_cls in PLUGINS:
    plugin = plugin_cls()
    print(f"Plugin: {plugin_cls.__name__}")
    for param_name in plugin.spec_dict.keys():
        # For demonstration, create a text box for each parameter
        value = plugin.spec_dict[param_name]
        input_widget = widgets.Text(
            value=str(value),
            description=param_name
        )
        display(input_widget)

Plugin: AdhesionFlexPlugin


Text(value='1', description='neighbor_order')

Text(value='[]', description='molecules')

Text(value='{}', description='densities')

Text(value='{}', description='binding_formulas')

...

Text(value='{}', description='field_specs')

Plugin: SurfacePlugin


Text(value='{}', description='params')

Plugin: VolumePlugin


Text(value='{}', description='params')

In [4]:
from cc3d.core.PyCoreSpecs import PLUGINS
print([p.__name__ for p in PLUGINS])

['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'CellTypePlugin', 'CenterOfMassPlugin', 'ChemotaxisPlugin', 'ConnectivityGlobalPlugin', 'ConnectivityPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'MomentOfInertiaPlugin', 'NeighborTrackerPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'SurfacePlugin', 'VolumePlugin']


In [ ]:
# Option 2
from Spec_UI import SpecificationSetupUI
ui = SpecificationSetupUI()
# Don't currently have a spepare Metadata class.
# User configuration still in JSON? (yes) or a list like "spec"
# Lots of constraints - initliazer email okie extracting dependacy relationships from the API
'''specs = [
    Metadata(),
    PottsCore(dim_x=100, dim_y=100, neighbor_order=2)
]'''

'specs = [\n    Metadata(), \n    PottsCore(dim_x=100, dim_y=100, neighbor_order=2)\n]'

In [6]:
from Spec_UI import PottsWidget, DEFAULTS
from IPython.display import display

potts_widget = PottsWidget(DEFAULTS["PottsCore"])
display(potts_widget.create_ui())

In [7]:
from Spec_UI import CellTypeWidget, DEFAULTS
from IPython.display import display

celltype_widget = CellTypeWidget(DEFAULTS["CellType"])
display(celltype_widget.create_ui())

In [8]:
ui.save_to_json()  # Manual save
ui.reset_all()     # Reset everything (pass None as required argument)
config = ui.current_config()  # Get current state

Validation error for AdhesionFlexPlugin: Could not validate single instance of Potts (AdhesionFlex)
Could not validate single instance of CellType (AdhesionFlex)
Validation error for BoundaryPixelTrackerPlugin: Could not validate single instance of Potts (BoundaryPixelTracker)
Could not validate single instance of CellType (BoundaryPixelTracker)
Validation error for ChemotaxisPlugin: Could not validate single instance of Potts (Chemotaxis)
Could not validate single instance of CellType (Chemotaxis)
Validation error for ContactPlugin: Could not validate single instance of Potts (Contact)
Could not validate single instance of CellType (Contact)
Validation error for CurvaturePlugin: Could not validate single instance of Potts (Curvature)
Could not validate single instance of CellType (Curvature)
Validation error for ExternalPotentialPlugin: Could not validate single instance of Potts (ExternalPotential)
Could not validate single instance of CellType (ExternalPotential)
Validation error fo